In [1]:
import torch
from math import factorial
from pytorch3d.io import load_obj

In [2]:
m = load_obj("model_normalized.obj",load_textures=False)

In [3]:
def M_ijk_torch(f_matrix,v_matrix,max_m):
    num_faces = len(f_matrix)
    
    # Extract coordinates
    ABC = v_matrix[f_matrix]
    # Calculate Determinants
    dets = torch.det(ABC)
    
    # Allocate Tensors
    M_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float64)
    C_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float64)
    D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float64)
    S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float64)
    
    # Calculate C Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        C_tensor[:,i,j,k] = (ABC[:,2][:,0]**i)*(ABC[:,2][:,1]**j)*(ABC[:,2][:,2]**k)*(factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))
                        
    # Calculate D Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # D_ijk=0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # D_ijk = 1
                            D_tensor[:,i,j,k] = 1
                        else:
                            D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
    # Calculate S Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # S_ijk = 0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # S_ijk = 1
                            S_tensor[:,i,j,k] = 1
                        else:
                            S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
    # Calculate M Tensor, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))*(dets[:]*S_tensor[:,i,j,k])
                    
    return torch.sum(M_tensor,0)
    

# M_ijk_torch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),3)

In [4]:
def moment_loss_torch(f_matrix_1, v_matrix_1, f_matrix_2, v_matrix_2, max_m = 10):
    return torch.linalg.norm(M_ijk_torch(f_matrix_1, v_matrix_1,max_m)-M_ijk_torch(f_matrix_2, v_matrix_2,max_m))

In [5]:
moment_loss_torch(m[1].verts_idx,m[0],m[1].verts_idx,m[0],10)

tensor(0., dtype=torch.float64)